**Install Paddle Paddle**

Official website: https://www.paddlepaddle.org.cn/

Github repo from paper: https://github.com/XinJCheng/CSPN/tree/master/cspn_paddle


In [13]:
!pip install gdown

In [14]:
!gdown https://drive.google.com/uc?id=1YqkbflmadV_dLuKcpMlqoy-5mCrKmWDy

Downloading...
From: https://drive.google.com/uc?id=1YqkbflmadV_dLuKcpMlqoy-5mCrKmWDy
To: /content/CSPN/cspn_paddle/paddlepaddle_gpu-0.0.0-cp36-cp36m-linux_x86_64.whl
149MB [00:00, 171MB/s]


In [15]:
!dir

demo.py  paddlepaddle_gpu-0.0.0-cp36-cp36m-linux_x86_64.whl  README.md


In [16]:
!python -m pip install paddlepaddle_gpu-0.0.0-cp36-cp36m-linux_x86_64.whl

**Verify correct installation of paddle paddle**

In [17]:
import paddle.fluid
paddle.fluid.install_check.run_check()

Running Verify Fluid Program ... 
Your Paddle Fluid works well on SINGLE GPU or CPU.
Your Paddle Fluid works well on MUTIPLE GPU or CPU.
Your Paddle Fluid is installed successfully! Let's start deep Learning with Paddle Fluid now


**Clone CSPN github repository**

In [18]:
!git clone https://github.com/XinJCheng/CSPN.git
%cd ./CSPN/cspn_paddle

Cloning into 'CSPN'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 211 (delta 78), reused 103 (delta 47), pack-reused 68
Receiving objects: 100% (211/211), 304.01 KiB | 841.00 KiB/s, done.
Resolving deltas: 100% (103/103), done.
/content/CSPN/cspn_paddle/CSPN/cspn_paddle


**Run the demo for 2d cspn module.**


In [22]:
!python demo.py --dimNum=2

 You could build paddle WITH_AVX=ON to get better performance.
The original error is: No module named 'paddle.fluid.core_avx'
W1126 10:17:24.311373   436 init.cc:162] AVX is available, Please re-compile on local machine
usage: demo.py [-h] [--dimNum {2,3}] [--featChan FEATCHAN] [--propKernel {3}]
               [--propStep PROPSTEP]
demo.py: error: argument --dimNum: invalid choice: '2' (choose from 2, 3)


**Run the demo for 3d cspn module.**

In [23]:
!python demo.py

 You could build paddle WITH_AVX=ON to get better performance.
The original error is: No module named 'paddle.fluid.core_avx'
W1126 10:17:27.883081   450 init.cc:162] AVX is available, Please re-compile on local machine
name: "cspn_affinity_propagate.tmp_23"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: -1
      dims: 1
      dims: 48
      dims: 64
      dims: 128
    }
    lod_level: 0
  }
}
persistable: false

W1126 10:17:28.462036   450 device_context.cc:259] Please NOTE: device: 0, CUDA Capability: 37, Driver API Version: 10.1, Runtime API Version: 9.0
W1126 10:17:28.465714   450 device_context.cc:267] device: 0, cuDNN Version: 7.6.
An exception was thrown!
 Invoke operator fill_constant error.
Python Callstacks: 
  File "/usr/local/lib/python3.6/dist-packages/paddle/fluid/framework.py", line 1844, in _prepend_op
    attrs=kwargs.get("attrs", None))
  File "/usr/local/lib/python3.6/dist-packages/paddle/fluid/initializer.py", line 189, in _